In [1]:
# Monte-Carlo playoff odds
# Generate my own playoff odds

# For now, I'm focusing on the mechanics of the simulation, and less so on the inputs (e.g., the projected team quality)
# So I'm using 538's win probabilities for each game, rather than computing my own

# I'm also using 538's results/schedule data, because it is so easy to use

import pandas as pd
import numpy as np

In [2]:
# Read in the 538 dataset, which has a row for each game in the current season (played or unplayed)
gms = pd.read_csv('../data/538/mlb-elo/mlb_elo_latest.csv')
gms

,date,season,neutral,playoff,team1,team2,elo1_pre,elo2_pre,elo_prob1,elo_prob2,...,pitcher1_rgs,pitcher2_rgs,pitcher1_adj,pitcher2_adj,rating_prob1,rating_prob2,rating1_post,rating2_post,score1,score2
0,2021-10-03,2021,0,NaN,ATL,NYM,1556.061995,1486.538702,0.631432,0.368568,...,58.278107,54.245129,23.976540,14.429515,0.608903,0.391097,NaN,NaN,NaN,NaN
1,2021-10-03,2021,0,NaN,STL,CHC,1538.522333,1448.501418,0.658442,0.341558,...,52.293738,46.314494,7.889568,3.349706,0.642413,0.357587,NaN,NaN,NaN,NaN
2,2021-10-03,2021,0,NaN,SEA,ANA,1507.363792,1475.740561,0.579371,0.420629,...,50.489752,NaN,7.908674,NaN,0.546707,0.453293,NaN,NaN,NaN,NaN
3,2021-10-03,2021,0,NaN,LAD,MIL,1600.568471,1545.521105,0.611835,0.388165,...,58.417971,56.160037,11.880021,0.125749,0.634612,0.365388,NaN,NaN,NaN,NaN
4,2021-10-03,2021,0,NaN,KCR,MIN,1472.094994,1482.146490,0.520063,0.479937,...,41.879897,43.950882,-36.551138,-27.088053,0.489689,0.510311,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2424,2021-04-01,2021,0,NaN,PHI,ATL,1499.854980,1537.334830,0.480611,0.519389,...,56.982418,53.435208,25.593166,-0.900219,0.537195,0.462805,1507.143856,1532.557621,3.0,2.0
2425,2021-04-01,2021,0,NaN,CHC,PIT,1510.606166,1466.885465,0.596242,0.403758,...,56.657868,47.268712,35.374576,-2.657647,0.687642,0.312358,1504.178744,1435.619713,3.0,5.0
2426,2021-04-01,2021,0,NaN,MIL,MIN,1502.386929,1522.960488,0.504931,0.495069,...,57.894228,57.914464,34.030150,17.163499,0.533019,0.466981,1518.710560,1536.695871,6.0,5.0
2427,2021-04-01,2021,0,NaN,DET,CLE,1450.437505,1513.164945,0.444496,0.555504,...,47.172259,61.318392,-0.833903,43.767481,0.378466,0.621534,1447.667412,1511.947362,3.0,2.0


In [3]:
gms.columns

Index(['date', 'season', 'neutral', 'playoff', 'team1', 'team2', 'elo1_pre',
       'elo2_pre', 'elo_prob1', 'elo_prob2', 'elo1_post', 'elo2_post',
       'rating1_pre', 'rating2_pre', 'pitcher1', 'pitcher2', 'pitcher1_rgs',
       'pitcher2_rgs', 'pitcher1_adj', 'pitcher2_adj', 'rating_prob1',
       'rating_prob2', 'rating1_post', 'rating2_post', 'score1', 'score2'],
      dtype='object')

In [4]:
# Split out the games that have been played vs those remaining
played = gms.dropna(subset=['score1']) # games that have a score
remain = gms.loc[gms.index.difference(played.index)] # all other games
played.shape, remain.shape

((2336, 26), (93, 26))

# Define some functions that will be used in the simulation

In [5]:
def compute_standings(gms_played):
    margins = gms_played['score1']-gms_played['score2']
    winners = pd.Series(np.where(margins>0, gms_played['team1'], gms_played['team2']))
    losers  = pd.Series(np.where(margins<0, gms_played['team1'], gms_played['team2']))
    standings = pd.concat([winners.value_counts().rename('W'), losers.value_counts().rename('L')], axis=1)
    return standings

compute_standings(played)

,W,L
SFG,102,54
LAD,100,56
TBD,97,59
MIL,94,62
HOU,91,65
NYY,89,67
CHW,88,68
BOS,88,68
STL,87,69
TOR,87,69


In [6]:
#  Create a data frame with the league/division mappings, to use to determine playoff berths
divisions = pd.DataFrame({
'SFG': ['N','NW'],
'LAD': ['N','NW'],
'TBD': ['A','AE'],
'MIL': ['N','NC'],
'HOU': ['A','AW'],
'CHW': ['A','AC'],
'BOS': ['A','AE'],
'NYY': ['A','AE'],
'TOR': ['A','AE'],
'OAK': ['A','AW'],
'SEA': ['A','AW'],
'SDP': ['N','NW'],
'ATL': ['N','NE'],
'CIN': ['N','NC'],
'PHI': ['N','NE'],
'STL': ['N','NC'],
'NYM': ['N','NE'],
'ANA': ['A','AW'],
'CLE': ['A','AC'],
'DET': ['A','AC'],
'CHC': ['N','NC'],
'COL': ['N','NW'],
'KCR': ['A','AC'],
'MIN': ['A','AC'],
'FLA': ['N','NE'],
'WSN': ['N','NE'],
'TEX': ['A','AW'],
'PIT': ['N','NC'],
'BAL': ['A','AE'],
'ARI': ['N','NW']
 }).T

divisions.columns = ['lg', 'div']
divisions

,lg,div
SFG,N,NW
LAD,N,NW
TBD,A,AE
MIL,N,NC
HOU,A,AW
CHW,A,AC
BOS,A,AE
NYY,A,AE
TOR,A,AE
OAK,A,AW


In [7]:

def sim_rem_games(remain):
    # Generate a random number for each game
    randoms = pd.Series(np.random.rand(len(remain)))
    randoms.index = remain.index

    # Figure out the winners and losers
    winners = pd.Series(np.where(randoms<remain['rating_prob1'], remain['team1'], remain['team2']))
    losers = pd.Series(np.where(randoms>remain['rating_prob1'], remain['team1'], remain['team2']))

    # Compute and return the standings
    standings = pd.concat([winners.value_counts().rename('W'), losers.value_counts().rename('L')], axis=1).fillna(0)
    for col in standings.columns: # convert to int
        standings[col] = standings[col].apply(int)
    return standings

sim_rem_games(remain)

,W,L
LAD,5,1
FLA,5,2
CIN,5,1
BOS,5,1
ANA,5,1
SFG,5,1
CLE,5,2
TOR,4,2
HOU,4,2
CHC,3,3


In [8]:
cur_standings = compute_standings(played)
rem_standings = sim_rem_games(remain)
full_standings = cur_standings+rem_standings
full_standings

,W,L
ANA,76,86
ARI,53,109
ATL,87,74
BAL,51,111
BOS,93,69
CHC,68,94
CHW,92,70
CIN,84,78
CLE,80,82
COL,75,86


In [9]:
# find playoff teams
def find_playoff_teams(standings):
    standings['wpct'] = standings['W'] / (standings['W'] + standings['L'])

    # Merge in the div/lg data
    standings['div'] = divisions['div']
    standings['lg'] = divisions['lg']

    # Rather than model out all the tie-breakers, I'm assuming that they are all random (not exactly true, but close enough),
    # and so I'm just generating a random number for each team, and we break ties by comparing that random num for each of the tied teams.
    # This is *so* much simpler and faster than modeling all the different scenarios.
    # It might be worth modeling them out with 1-2 days left in the season, but for most of the season, I way prefer using the random num to break ties
    standings['rand'] = np.random.rand(len(standings))

    # Now sort, and break ties using the rand
    sorted = standings.sort_values(by=['wpct', 'rand'], ascending=False)

    div_winners =  list(sorted.groupby(['div']).head(1).index.values)

    wc_contenders = sorted[~sorted.index.isin(div_winners)]
    wc_winners = list(wc_contenders.groupby(['lg']).head(2).index.values)


    return div_winners + wc_winners

     

full_standings, find_playoff_teams(full_standings)

(       W    L      wpct div lg      rand
 ANA   76   86  0.469136  AW  A  0.677065
 ARI   53  109  0.327160  NW  N  0.585973
 ATL   87   74  0.540373  NE  N  0.155326
 BAL   51  111  0.314815  AE  A  0.517711
 BOS   93   69  0.574074  AE  A  0.503218
 CHC   68   94  0.419753  NC  N  0.662898
 CHW   92   70  0.567901  AC  A  0.698468
 CIN   84   78  0.518519  NC  N  0.211526
 CLE   80   82  0.493827  AC  A  0.631513
 COL   75   86  0.465839  NW  N  0.827221
 DET   78   84  0.481481  AC  A  0.227151
 FLA   66   96  0.407407  NE  N  0.708921
 HOU   94   68  0.580247  AW  A  0.179373
 KCR   74   88  0.456790  AC  A  0.750877
 LAD  102   60  0.629630  NW  N  0.092588
 MIL   96   66  0.592593  NC  N  0.252730
 MIN   73   89  0.450617  AC  A  0.187639
 NYM   75   87  0.462963  NE  N  0.202930
 NYY   95   67  0.586420  AE  A  0.439382
 OAK   88   74  0.543210  AW  A  0.800722
 PHI   86   76  0.530864  NE  N  0.404050
 PIT   62  100  0.382716  NC  N  0.314283
 SDP   80   82  0.493827  NW  N  0

In [10]:
%%prun -s cumulative # This runs the code profiler, which creates data I can use to find opportunities for me to speed up the code

[find_playoff_teams(full_standings) for _ in range(1000)]
None # This is to suppress printing the output, which is 1000 lines of the same list of teams

         8259896 function calls (8190896 primitive calls) in 3.905 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    3.905    3.905 {built-in method builtins.exec}
        1    0.000    0.000    3.905    3.905 <string>:3(<module>)
        1    0.010    0.010    3.905    3.905 <string>:3(<listcomp>)
     1000    0.034    0.000    3.895    0.004 <ipython-input-9-d4cf20c5399b>:2(find_playoff_teams)
     2000    0.013    0.000    1.162    0.001 groupby.py:2764(head)
     1000    0.011    0.000    1.038    0.001 frame.py:5417(sort_values)
    12000    0.032    0.000    1.013    0.000 frame.py:2987(__getitem__)
     1000    0.030    0.000    0.736    0.001 sorting.py:264(lexsort_indexer)
     3000    0.006    0.000    0.673    0.000 frame.py:3049(_getitem_bool_array)
     4000    0.022    0.000    0.673    0.000 managers.py:1454(take)
     4000    0.010    0.000    0.663    0.000 frame.py:3147(__set

In [11]:
def finish_one_season(incoming_standings, remain):
    rem_standings = sim_rem_games(remain)
    full_standings = incoming_standings+rem_standings
    playoff_teams = find_playoff_teams(full_standings)
    return playoff_teams

finish_one_season(cur_standings, remain)

['SFG', 'TBD', 'MIL', 'HOU', 'CHW', 'ATL', 'LAD', 'BOS', 'STL', 'TOR']

In [12]:
#
def sim_n_seasons(incoming_standings, remain, n):
    # The first 6 teams in the list are div winners, the rest are WCs
    return pd.DataFrame([{'sim': i, 'finish': 'div' if t[0] < 6 else 'wc', 'tm': t[1]}  
        for i in range(n) 
        for t in enumerate(finish_one_season(incoming_standings, remain)) 
        ])

sim_results = sim_n_seasons(cur_standings, remain, 10)
sim_results

,sim,finish,tm
0,0,div,SFG
1,0,div,MIL
2,0,div,TBD
3,0,div,HOU
4,0,div,CHW
...,...,...,...
95,9,div,CHW
96,9,wc,LAD
97,9,wc,BOS
98,9,wc,STL


In [13]:
# Count the number of div/wc/playoff appearances by team from a set of results
def summarize_sim_results(df_results):
    summary = df_results[['tm', 'finish']].value_counts().unstack().fillna(0)
    for col in summary.columns:
        summary[col] = summary[col].apply(int)
    summary['playoffs'] = summary['div'] + summary['wc']
    return summary

summarize_sim_results(sim_results)

finish,div,wc,playoffs
tm,,,
ATL,9,0,9
BOS,0,7,7
CHW,10,0,10
HOU,10,0,10
LAD,1,9,10
MIL,10,0,10
NYY,0,9,9
PHI,1,0,1
SEA,0,1,1


In [14]:
summarize_sim_results(sim_n_seasons(cur_standings, remain, 10))

finish,div,wc,playoffs
tm,,,
ATL,6,0,6
BOS,0,9,9
CHW,10,0,10
HOU,10,0,10
LAD,2,8,10
MIL,10,0,10
NYY,0,6,6
PHI,4,0,4
SFG,8,2,10


# Now put it all together and simulate a large number of seasons
## (Possibly multiple times, for comparison)


In [15]:
#%%prun  -s cumulative

# We want to simulate the rest of the season a large number of times (e.g., 1000 or 100K or more)
# We also want to run multiple of these sets, to observe the variation across different runs
# So we run trials of num_seasons seasons, num_trials times

NUM_SEASONS = 10*1000
NUM_TRIALS = 5

# Collect just the number of playoff appearances for now, since we'll compare playoff appearance percentagees
def run_trial(incoming_standings, remaining_gms, num_seasons):
    return summarize_sim_results(sim_n_seasons(incoming_standings, remaining_gms, num_seasons))['playoffs']

# Run the sims
totals = pd.concat([run_trial(cur_standings, remain, NUM_SEASONS) for _ in range(NUM_TRIALS)], axis=1)

# Fill in blanks (and convert to ints)
totals = totals.fillna(0).apply(lambda col: col.apply(int))

totals

,playoffs,playoffs,playoffs,playoffs,playoffs
tm,,,,,
ATL,9367,9397,9392,9358,9402
BOS,7766,7761,7716,7776,7732
CHW,10000,10000,10000,10000,10000
HOU,9991,9993,9991,9991,9989
LAD,10000,10000,10000,10000,10000
MIL,10000,10000,10000,10000,10000
NYY,7556,7610,7503,7502,7631
OAK,56,49,53,53,50
PHI,633,603,608,642,598


In [16]:
# Average across the trials, to compute the playoff appearance frequency
(totals.apply(np.mean, axis=1)/NUM_SEASONS).sort_values(ascending=False)

tm
CHW    1.00000
LAD    1.00000
MIL    1.00000
SFG    1.00000
TBD    1.00000
STL    0.99990
HOU    0.99910
ATL    0.93832
BOS    0.77502
NYY    0.75604
TOR    0.40288
SEA    0.06174
PHI    0.06168
OAK    0.00522
CIN    0.00010
dtype: float64

In [17]:
# Which teams have the widest range?  How big is that range?
((totals.apply(max, axis=1) - totals.apply(min, axis=1))/NUM_SEASONS).sort_values(ascending=False)

tm
NYY    0.0129
TOR    0.0112
SEA    0.0073
BOS    0.0060
ATL    0.0044
PHI    0.0044
OAK    0.0007
HOU    0.0004
STL    0.0004
CIN    0.0004
CHW    0.0000
LAD    0.0000
MIL    0.0000
SFG    0.0000
TBD    0.0000
dtype: float64